# Transformaciones con Tipos de Datos Complejos: Arrays, Structs y Maps

En este notebook:
- Trabajaremos con columnas tipo array y struct a partir del dataset MovieLens.
- Aplicaremos funciones como `explode`, y crearemos structs de varias columnas.


In [ ]:
from pyspark.sql.functions import split, explode, struct, col

# Dataset MovieLens: queremos ver para cada usuario, la lista de películas que calificó
ratings = spark.read.csv("/dbfs/FileStore/movielens/ratings.csv", header=True, inferSchema=True)
movies = spark.read.csv("/dbfs/FileStore/movielens/movies.csv", header=True, inferSchema=True)

# Agrupar calificaciones por usuario (array de movieIds)
user_movies = ratings.groupBy("userId").agg(collect_list("movieId").alias("peliculas_calificadas"))
user_movies.show(5, truncate=False)

# Explode: expandir para tener una fila por usuario/película
user_movies_exploded = user_movies.withColumn("pelicula_id", explode(col("peliculas_calificadas")))
user_movies_exploded.show(5)

# Struct: juntar nombre y género de la película
movies_struct = movies.withColumn("info", struct("title", "genres"))
movies_struct.select("movieId", "info").show(5, truncate=False)

# BONUS: Crear columna tipo map (clave=movieId, valor=rating) para cada usuario
from pyspark.sql.functions import create_map, lit
# Ejemplo sencillo: para cada usuario, tomar sus primeras 3 calificaciones y armar el map
sample = ratings.filter(col("userId") == 1).limit(3)
sample_map = sample.withColumn("movie_rating_map", create_map(col("movieId"), col("rating")))
sample_map.show()
